In [1]:
import kplr


        #fits1=lc1[i].open()
       # fname1=str(2305930)+'_'+str(i)+'.fits'
    #fits1.writeto(fname1)
     #   fits1.close()



client = kplr.API()
koi = client.koi(1274.01)

/home/zapredfield/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
lcs1 = koi.get_light_curves()

In [3]:
teff=[]
logg=[]
feh=[]


time1, flux1, ferr1 = [], [], []
for lc in lcs1:
    with lc.open() as f:

        hdu_data = f[1].data
        header=f[0].header
        time1.append(hdu_data["time"]) #obtenemos los tiempos
        flux1.append(hdu_data["pdcsap_flux"]) #flujos
        ferr1.append(hdu_data["pdcsap_flux_err"]) #errores de flujo
teff.append(header["TEFF"]) #cargamos la tempertaura efectiva 
logg.append(header["LOGG"])#cargamos la log(g)
feh.append(header["FEH"]) #cargamos la metalicidad

In [4]:
print teff,logg,feh

[5286] [4.568] [-0.26]


In [5]:
#Para estrella 1
import matplotlib.pyplot as plt
import numpy


def nan(t,f,err): #Eliminar NANS
    tf=[]
    ff=[]
    errf=[]
    for i in range(0,len(f)):
        if numpy.isnan(f[i])==False:
            tf.append(t[i])
            ff.append(f[i])
            errf.append(err[i])
    return tf,ff,errf


def unirt(listas): #Función simple para unir listas
    final=[]
    for lista in listas:
        for dato in lista:
            final.append(dato)
    return final


def unirf(listas): #Función que resta las medias de cada quarter y une listas
    f=[]
    for lista in listas:
        mean=numpy.mean(lista)
        for dato in lista:
            f.append(dato-mean)
    return f


In [6]:
#Primer objeto
#Primero se remueven NANs
tsinnan=[]
fsinnan=[]
errsinnan=[]
for i in range(0,len(time1)):
    ttemp,ftemp,ferrtemp=nan(time1[i],flux1[i],ferr1[i])
    errsinnan.append(ferrtemp)
    tsinnan.append(ttemp)
    fsinnan.append(ftemp)   

t1=unirt(tsinnan) #Unimos quarters de tiempo
f1=np.array(unirf(fsinnan)) #unimos flujos normalizando
ferr1=unirt(errsinnan) #unimos quarters de error del flujo 

In [48]:
import scipy.signal as MF

f1filt=MF.medfilt(f1,11)
for i in range(0,50):
    f1filt=MF.medfilt(f1filt1,11)



#f1filt3=MF.medfilt(f1,11)

In [49]:
import matplotlib.pyplot as plt
#Funciones de ploteo
plt.rcParams['figure.figsize'] = (20, 5)
plt.title("KIC 2305930")
plt.plot(f1filt,'r')
plt.show()


In [52]:
from gatspy.periodic import LombScargleFast


model = LombScargleFast().fit(t1,f1filt,ferr1)
periods, power = model.periodogram_auto(nyquist_factor=5)



In [54]:
plt.rcParams['figure.figsize'] = (10, 5)
plt.plot(periods, power,linewidth=3)
plt.xlabel("days")
plt.xlim(0,80)
plt.ylabel("Lomb-Scargle Power")
plt.show()